In [1]:
import os
import re
import time
import csv
import json
import random
import requests
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options

In [3]:
pip install selenium==3.141.0

     ---------------------------------------- 0.0/904.6 kB ? eta -:--:--
     ---------------------------------------- 10.2/904.6 kB ? eta -:--:--
     - ----------------------------------- 41.0/904.6 kB 495.5 kB/s eta 0:00:02
     ----- -------------------------------- 122.9/904.6 kB 1.0 MB/s eta 0:00:01
     --------------------- ---------------- 512.0/904.6 kB 2.9 MB/s eta 0:00:01
     -------------------------------------- 904.6/904.6 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: selenium
    Found existing installation: selenium 3.14.0
    Uninstalling selenium-3.14.0:
      Successfully uninstalled selenium-3.14.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install webdriver-manager

  Obtaining dependency information for webdriver-manager from https://files.pythonhosted.org/packages/b1/51/b5c11cf739ac4eecde611794a0ec9df420d0239d51e73bc19eb44f02b48b/webdriver_manager-4.0.1-py2.py3-none-any.whl.metadata
  Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl.metadata (12 kB)
Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl (27 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
chrome_options = webdriver.ChromeOptions()
chrome_options.headless = False
browser = webdriver.Chrome(executable_path = 'C:\\Users\\BVB\\Downloads\\DMWEBprj1\\chromedriver',
                          options = chrome_options)
url_head = 'https://www.estimize.com/users/sign_in'
browser.get(url_head)
time.sleep(random.uniform(1,5))

In [6]:
import time
from selenium import webdriver

def sign_in(username, password):
    url = 'https://www.estimize.com/users/sign_in'
    browser = webdriver.Chrome()
    browser.get(url)
    email_field = browser.find_element_by_id('user_login')
    password_field = browser.find_element_by_id('user_password')
    submit_btn = browser.find_element_by_xpath('//input[@type="submit"]')
    email_field.send_keys(username)
    password_field.send_keys(password)
    submit_btn.click()
    time.sleep(1)
    browser.quit()
username = 'bharathbv8@gmail.com'
password = 'qwerty123asdzxc'
sign_in(username, password)


In [7]:
sign_in()

In [8]:
tickers=["AMZN","AAPL","MSFT","GOOG","TSLA","JNJ","PG","NVDA","CSCO","BABA","HD", 
"BIDU","WMT","CRM","LULU","TGT","PANW","ADBE","VMW","MU","NKE","ORCL","BB","HPQ","COST","AMAT", 
"BAC","CVX","AMGN","PG"]

quarters = ['fq1-2020', 'fq2-2020', 'fq3-2020', 'fq4-2020']

In [9]:
def scrape_overview_data(tickers, quarters, browser):
    overview_all = {}

    try:
        for ticker in tickers:
            for quarter in quarters:
                url = f'https://www.estimize.com/{ticker}/{quarter}?metric_name=eps&chart=historical'
                browser.get(url)
                time.sleep(1)
                overview = {
                    'quarter': quarter,
                    'Ticker': browser.find_element_by_class_name('release-header-information-title').text,
                    'Company Name': browser.find_element_by_class_name('release-header-information-description').text
                }
                sectors = browser.find_element_by_class_name('release-header-information-breadcrumb')
                overview['Sectors'] = sectors.text.split('›')[0]
                overview['Industries'] = sectors.text.split('›')[1]
                numbers = browser.find_elements_by_class_name('summary-sub-title')
                overview['Number of Followers'] = numbers[0].text
                overview['Number of Analysts'] = numbers[1].text
                overview_all[len(overview_all)] = overview
    except Exception as e:
        print(f"An error occurred: {e}")

    return overview_all



In [10]:
company_info = scrape_overview(browser)

with open('Company.json', 'w') as outfile:
    json.dump(company_info, outfile, indent=4)
    
df=pd.DataFrame(company_info)
df_trans = df.T
df_trans.to_csv('Company.csv')

In [11]:
def scrape_eps(browser):
    eps_data = {}

    for ticker in tickers:
        for quarter in quarters:
            url = f'https://www.estimize.com/{ticker}/{quarter}?metric_name=eps&chart=historical'
            browser.get(url)
            time.sleep(1)
            analysts_generic = browser.find_elements_by_class_name('estimates-tbl-consensus-column')
            values_generic = browser.find_elements_by_class_name('estimates-tbl-consensus-eps')
            for generic_count in range(len(analysts_generic)):
                eps_generic = {
                    'quarter': quarter,
                    'Ticker': ticker,
                    'Company Name': browser.find_element_by_class_name('release-header-information-title').text,
                    'Name': analysts_generic[generic_count].text.split('\n')[0],
                    'Type': "Generic",
                    'Estimated Value': values_generic[generic_count].text
                }
                eps[len(eps_data)] = eps_generic
            analysts = browser.find_elements_by_class_name('username')
            analysts_values = browser.find_elements_by_class_name('estimates-tbl-eps')

            for analyst_count in range(len(analysts)):
                eps_analyst = {
                    'quarter': quarter,
                    'Ticker': ticker,
                    'Company Name': browser.find_element_by_class_name('release-header-information-title').text,
                    'Name': analysts[analyst_count].text,
                    'Type': "Analyst",
                    'Estimated Value': analysts_values[analyst_count].text
                }
                eps[len(eps_data)] = eps_analyst
    return eps_data


In [12]:
eps_info = scrape_eps(browser)

with open('EPS.json', 'w') as outfile:
    json.dump(eps_info, outfile, indent=4)

df=pd.DataFrame(eps_info)
df_trans = df.T
df_trans.to_csv('EPS.csv')

In [13]:
def Remove_dup(duplicate):
    final_list = []
    
    for num in range(len(duplicate)):
        if duplicate[num] not in final_list:
            final_list.append(duplicate[num])
    return final_list

In [14]:
def scrape_analysts_list(tickers, quarters, browser):
    analysts_list = {}

    for ticker in tickers:
        for quarter in quarters:
            url = f'https://www.estimize.com/{ticker}/{quarter}?metric_name=eps&chart=historical'
            browser.get(url)
            time.sleep(1)
            links = browser.find_elements_by_tag_name("a")
            for link in links:
                href = link.get_attribute('href')
                if href and '/users/' in href and 'sign_out' not in href:
                    analysts_list[len(analysts_list)] = href

    ana_list_final = Remove_dup(ana_list)
    print (len(ana_list_final))

    return ana_list_final


In [15]:
def scrape_analysts_info(browser):
    analysts_urls = scrape_analysts_list(browser)
    analyst_profile_metrics = ['Error rate', 'Accuracy Percentile', 'Points', 'Points/Estimate', 'Stocks', 'Pending']
    analysts_overview_final = {}

    for analyst_url in analysts_urls:
        browser.get(analyst_url)
        time.sleep(1)

        analyst_overview = {}
        try:
            analyst_overview['Name'] = browser.find_element_by_class_name('profile-display-name').text
            analyst_overview['User ID'] = browser.find_element_by_class_name('profile-username').text
            analyst_overview['Roles'] = browser.find_element_by_class_name('profile-bio-categorizations').text

            date_info = browser.find_element_by_class_name('profile-activity-stats').text.split(' ')
            analyst_overview['Join Date'] = date_info[2] + ' ' + date_info[3]
            analyst_overview['Analyst Confidence Score'] = browser.find_element_by_class_name('value').text

            profile_stats = browser.find_elements_by_class_name('profile-stat')
            for i in range(len(profile_stats)):
                analyst_overview[analyst_profile_metrics[i]] = profile_stats[i].text

            analysts_overview_final[len(analysts_overview_final) + 1] = analyst_overview
        except Exception as e:
            # Handle exceptions (you might want to log the exception or handle it differently)
            print(f"An error occurred: {e}")

    return analysts_overview_final


In [16]:
analysts_info = scrape_analysts_info(browser)

with open('Analysts_Info.json', 'w') as outfile:
    json.dump(analysts_info, outfile, indent=4)

df=pd.DataFrame(analysts_info)
df_trans = df.T
df_trans.to_csv('Analysts_Info.csv')

1504


In [19]:
import pandas as pd

# List of CSV file names
file_names = ['EPS2020.csv', 'EPS2021.csv', 'EPS2022.csv']

# Initialize an empty DataFrame to store the concatenated data
concatenated_df = pd.DataFrame()

# Loop through each file and concatenate its content to the DataFrame
for file_name in file_names:
    df = pd.read_csv(file_name)
    concatenated_df = pd.concat([concatenated_df, df], ignore_index=True)

# Write the concatenated DataFrame to a new CSV file
concatenated_df.to_csv('EPSDATA.csv', index=False)


In [20]:
import pandas as pd

# List of CSV file names
file_names = ['Company2020.csv', 'Company2021.csv', 'Company2022.csv']

# Initialize an empty DataFrame to store the concatenated data
concatenated_df = pd.DataFrame()

# Loop through each file and concatenate its content to the DataFrame
for file_name in file_names:
    df = pd.read_csv(file_name)
    concatenated_df = pd.concat([concatenated_df, df], ignore_index=True)

# Write the concatenated DataFrame to a new CSV file
concatenated_df.to_csv('COMPANYDATA.csv', index=False)


In [22]:
import pandas as pd

# List of CSV file names
file_names = ['AnalystsInfo2020.csv', 'AnalystsInfo2021.csv', 'AnalystsInfo2022.csv']

# Initialize an empty DataFrame to store the concatenated data
concatenated_df = pd.DataFrame()

# Loop through each file and concatenate its content to the DataFrame
for file_name in file_names:
    df = pd.read_csv(file_name)
    concatenated_df = pd.concat([concatenated_df, df], ignore_index=True)

# Write the concatenated DataFrame to a new CSV file
concatenated_df.to_csv('ANALYSTINFODATA.csv', index=False)
